# Earth Surface Temperature Analysis
----------
In this analysis I will be looking at the dataset _[Climate Change: Earth Surface Temperature Data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)_ by Berkley Earth. This dataset consists of temperature readings from around the world that go back to at least the 1760's in some areas and up until 2013.

*Note*: This analysis does not include the 95% confidence interval for temperatures.

This notebook is best viewed with [Jupyter nbviewer](https://nbviewer.jupyter.org/github/kylepollina/Surface_Temperature_Analysis/blob/master/Earth%20Surface%20Temperature%20Analysis.ipynb). The *Contents* links will not work on Github.

##### Contents
1. [Preface](#Preface)
2. [Global Land Temperatures By Country](#Global-Land-Temperatures-By-Country)
3. [Global Land Temperatures By Major City](#Global-Land-Temperatures-By-Major-City)
4. [Global Land Temperatures By State](#Global-Land-Temperatures-By-State)

    

##### Links
- Dataset: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data
- Notebook: [Jupyter nbviewer](https://nbviewer.jupyter.org/github/kylepollina/Surface_Temperature_Analysis/blob/master/Earth%20Surface%20Temperature%20Analysis.ipynb)
- Source: https://github.com/circulEarth/SurfaceTemperatures
- Author: https://github.com/kylepollina

## Preface
----------

1. [The data](#The-data)
2. [Helper functions](#Helper-functions)

### Packages
- [pandas](https://pandas.pydata.org/): Used for reading data into [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) objects
- [altair](https://altair-viz.github.io/): Used for plotting and interactive visualization
- [os](https://docs.python.org/3/library/os.html): Used for operating system behavior, specifically [walking](https://docs.python.org/3/library/os.html?highlight=os%20walk#os.walk) through the filesystem
- [datetime.date](https://docs.python.org/2/library/datetime.html#date-objects): Used for reading/parsing dates from strings using [datetime.date.fromisoformat()](https://docs.python.org/3/library/datetime.html#datetime.date.fromisoformat)

In [1]:
import pandas
import altair as alt
import os
from datetime import date

### The data
This dataset consists of five sources. I have downloaded them into a directory named `/data/BerkleyEarth`. The `datafiles` list below consists of the names of each of the five .csv files. 

In [2]:
datafiles = []

for directory, _, files in os.walk('data/BerkleyEarth'):
    for file in files:
        filename = os.path.join(directory, file)
        datafiles.append(filename)

print(datafiles)

[]


`print_datafiles()` - Orderly print each datafile and index within the `datafiles` list

In [3]:
def print_datafiles():
    for index, filename in enumerate(datafiles):
        print(str(index) + ". " + filename)
            
print_datafiles()

#### Temperatures by Country

In [4]:
# Load country data into DataFrame named `country_temps`
country_temps = pandas.read_csv('data/BerkleyEarth/GlobalLandTemperaturesByCountry.csv')

# Drop any rows with a NaN/Empty values for any of the attributes
country_temps = country_temps.dropna()

# Save each unique country name into a set
all_countries = set(country_temps['Country'])

# Display DataFrame
country_temps

FileNotFoundError: [Errno 2] File data/BerkleyEarth/GlobalLandTemperaturesByCountry.csv does not exist: 'data/BerkleyEarth/GlobalLandTemperaturesByCountry.csv'

#### Temperatures by State

In [ ]:
# Load state data into DataFrame named `state_temps`
state_temps = pandas.read_csv('data/BerkleyEarth/GlobalLandTemperaturesByState.csv')

# Drop any rows with a NaN/Empty value for any of the attributes
state_temps = state_temps.dropna()

# Save each unique state name into a set
all_states = set(state_temps['State'])

# Display DataFrame
state_temps

#### Temperature by Major City

In [ ]:
# Load state data into DataFrame named `major_city_temps`
major_city_temps = pandas.read_csv('data/BerkleyEarth/GlobalLandTemperaturesByMajorCity.csv')

# Drop any rows with a NaN/Empty value for any of the attributes
major_city_temps = major_city_temps.dropna()

# Save each unique major city name into a set
all_major_cities = set(major_city_temps['City'])

# Display DataFrame
major_city_temps

#### Temperature by City
The 'Temperature By City' dataset is much larger than the others and takes significantly more time to load. I have commented out the loading of the dataset to save run time, but left this in for possible future analysis of each city.

In [ ]:
# Load state data into DataFrame named `major_city_temps`
# city_temps = pandas.read_csv('data/BerkleyEarth/GlobalLandTemperaturesByCity.csv')

# Drop any rows with a NaN/Empty value for any of the attributes
# city_temps = city_temps.dropna() 

# Save each unique major city name into a set
# all_cities = set(city_temps['City'])

# Display DataFrame
# city_temps

### Helper functions

`yearly_temps(location)` - Returns a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) of average temperatures recorded over each year for given location.

Parameters:
- `location` - String of location, name of Country, State, or Major City
- `dataset` - Optional argument to force lookup in specific dataset. For example, `New York` appears in `state_temps` and `major_city_temps`. `dataset` can equal one of the following three values (defaults to `None` if no dataset specified):
    - `'Country'`
    - `'State'`
    - `'Major City'`

*examples* 

`new_york_temps = yearly_temps('New York', dataset = 'State')`

`new_york_city_temps = yearly_temps('New York', dataset = 'Major City')`

In [ ]:
def get_yearly_temps(location, dataset = None):
    
    # Search location in country dataset
    if location in all_countries and (dataset is None or (dataset == 'Country')):
        # Filter country_temps by given country
        filtered_temps = country_temps[country_temps['Country'] == location]
    
    # Search location in state dataset
    elif location in all_states and (dataset is None or (dataset == 'State')):
        # Filter state_temps by given state
        filtered_temps = state_temps[state_temps['State'] == location]
    
    # Search location in major city dataset
    elif location in all_major_cities and (dataset is None or (dataset == 'Major City')):
        # Filter major_city_temps by given city
        filtered_temps = major_city_temps[major_city_temps['City'] == location]
    
    # Location not found
    else:
        raise ValueError("Location not found in any loaded dataset")
        
    # Sort recorded temperatures by date
    filtered_temps = filtered_temps.sort_values(by = ['dt'])
    
    # Result data to be returned
    yearly_temps = {'Year': [], 'AverageTemperature': []}
    
    # Get year from ISO date
    first_recorded_year = date.fromisoformat(filtered_temps.iloc[0]['dt']).year
    current_year = first_recorded_year
    
    # Sum and total used for averaging
    sum_temps = 0.0
    total_dates_in_year = 0
    
    for index, row in filtered_temps.iterrows():
        # Convert the `dt` value into a Python Date object
        current_date = date.fromisoformat(row['dt'])
    
        # If we have switched over into a new year
        if current_date.year != current_year: 
            # Average temperatures over previous year
            yearly_avg_temp = sum_temps / total_dates_in_year
            
            # Add to result data
            yearly_temps['Year'].append(current_year)
            yearly_temps['AverageTemperature'].append(yearly_avg_temp)
            
            # Update current_year
            current_year = current_date.year
            
            # Reset sum/total
            sum_temps = 0.0
            total_dates_in_year = 0
    
        # Increase sum/total
        sum_temps += row['AverageTemperature']
        total_dates_in_year += 1
    
    # Convert resulting data into DataFrame and return
    return pandas.DataFrame(data = yearly_temps)
    

## Global Land Temperatures By Country
-------

1. [North America](#North-America)
2. [South America](#South-America)
3. [Europe](#Europe)
4. [Africa](#Africa)
5. [Asia](#Asia)
6. [Oceania](#Oceania)

### North America

In [ ]:
location = 'United States'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Canada'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Mexico'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'North America'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'North America'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### South America

In [ ]:
location = 'Brazil'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Chile'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Argentina'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Venezuela'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'South America'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Europe

In [ ]:
location = 'United Kingdom'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'France'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Germany'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Spain'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Sweden'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Europe'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Africa

In [ ]:
location = 'Egypt'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'South Africa'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Kenya'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Uganda'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Africa'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Asia

In [ ]:
location = 'China'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'India'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Russia'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Japan'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Asia'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Oceania

In [ ]:
location = 'Australia'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'New Zealand'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Papua New Guinea'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

In [ ]:
location = 'Oceania'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

## Global Land Temperatures By Major City
-------
1. [New York City](#New-York-City)
2. [Los Angeles](#Los-Angeles)
3. [Chicago](#Chicago)
4. [London](#London)
5. [Paris](#Paris)

### New York City

In [ ]:
location = 'New York'
temps = get_yearly_temps(location, dataset = 'Major City')

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Los Angeles

In [ ]:
location = 'Los Angeles'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Chicago

In [ ]:
location = 'Chicago'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### London

In [ ]:
location = 'London'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Paris

In [ ]:
location = 'Paris'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

## Global Land Temperatures By State
-------
1. [Illinois](#Illinois)
2. [California](#California)
3. [Florida](#Florida)
4. [Texas](#Texas)
5. [New York](#New-York)
    
Here I will be focusing on U.S. states, but states from other countries are included in the dataset.

### Illinois

In [ ]:
location = 'Illinois'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### California

In [ ]:
location = 'California'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Florida

In [ ]:
location = 'Florida'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### Texas

In [ ]:
location = 'Texas'
temps = get_yearly_temps(location)

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)

### New York

In [ ]:
location = 'New York'
temps = get_yearly_temps(location, dataset = 'State')

# Line plot using Altair
alt.Chart(temps).mark_line().encode(
    x = 'Year:O',
    y = alt.Y('AverageTemperature', scale=alt.Scale(zero=False))
).properties(
    title = 'Yearly Average Temperatures (C) of ' + location,
    width = 800
)